<a href="https://colab.research.google.com/github/Korniev/Machine-Learning-projects/blob/main/image_classification_tiny_imagenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import tensorflow as tf
from keras.applications import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
# upload the file to Google Colab
from google.colab import files
uploaded = files.upload()

# List the uploaded files to verify
for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')

# Extract the uploaded file
import zipfile
import os

# Assuming the uploaded file is named 'archive.zip'
zip_path = '/content/archive.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content')

# Verify the extracted files
!ls /content/tiny-imagenet-200

Saving archive.zip to archive.zip
Uploaded file: archive.zip
test  train  val  wnids.txt  words.txt


In [12]:
import os
import shutil

val_dir = '/content/tiny-imagenet-200/val'
val_images_dir = os.path.join(val_dir, 'images')
val_annotations_file = os.path.join(val_dir, 'val_annotations.txt')

# Create subdirectories for each class
with open(val_annotations_file, 'r') as f:
    for line in f.readlines():
        parts = line.strip().split('\t')
        image_name = parts[0]
        class_name = parts[1]

        class_dir = os.path.join(val_dir, class_name)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)

        source = os.path.join(val_images_dir, image_name)
        destination = os.path.join(class_dir, image_name)
        shutil.move(source, destination)

# Remove the now-empty images directory
shutil.rmtree(val_images_dir)


In [13]:


# Define paths
train_dir = './tiny-imagenet-200/train'
val_dir = './tiny-imagenet-200/val'

# Define image data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=64,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(64, 64),
    batch_size=64,
    class_mode='categorical'
)


Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [14]:
# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(200, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Fine-tune the entire model
for layer in base_model.layers:
    layer.trainable = True

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/20
1563/1563 [==============================] - 264s 150ms/step - loss: 3.1836 - accuracy: 0.2973 - val_loss: 2.5173 - val_accuracy: 0.4026 - lr: 1.0000e-04
Epoch 2/20
1563/1563 [==============================] - 228s 146ms/step - loss: 2.2674 - accuracy: 0.4510 - val_loss: 2.3698 - val_accuracy: 0.4373 - lr: 1.0000e-04
Epoch 3/20
1563/1563 [==============================] - 229s 147ms/step - loss: 2.0031 - accuracy: 0.5040 - val_loss: 2.2545 - val_accuracy: 0.4661 - lr: 1.0000e-04
Epoch 4/20
1563/1563 [==============================] - 226s 145ms/step - loss: 1.8316 - accuracy: 0.5362 - val_loss: 2.2548 - val_accuracy: 0.4715 - lr: 1.0000e-04
Epoch 5/20
1563/1563 [==============================] - 227s 145ms/step - loss: 1.6897 - accuracy: 0.5670 - val_loss: 2.2495 - val_accuracy: 0.4820 - lr: 1.0000e-04
Epoch 6/20
1563/1563 [==============================] - 228s 146ms/step - loss: 1.5781 - accuracy: 0.5924 - val_loss: 2.1610 - val_accuracy: 0.4937 - lr: 1.0000e-04
Epoch 7/20

In [15]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(val_generator)
print(f'Validation Accuracy: {val_accuracy}')

157/157 [==============================] - 5s 31ms/step - loss: 2.0875 - accuracy: 0.5212
Validation Accuracy: 0.5212000012397766
